In [1]:
from joblib import load

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [3]:
bull_valid = pd.read_csv('Valid.csv',parse_dates=['saledate'],low_memory=False)

In [4]:
bull_valid.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1222837,902859,1376,121,3,1000,0.0,NaN,2012-01-05,375L,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
1,1222839,1048320,36526,121,3,2006,4412.0,Medium,2012-01-05,TX300LC2,...,None or Unspecified,"12' 4""",None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN
2,1222841,999308,4587,121,3,2000,10127.0,Medium,2012-01-05,270LC,...,None or Unspecified,"12' 4""",None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1222843,1062425,1954,121,3,1000,4682.0,Low,2012-01-05,892DLC,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
4,1222845,1032841,4701,121,3,2002,8150.0,Medium,2012-01-04,544H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [5]:
bull_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 52 columns):
SalesID                     11573 non-null int64
MachineID                   11573 non-null int64
ModelID                     11573 non-null int64
datasource                  11573 non-null int64
auctioneerID                11573 non-null int64
YearMade                    11573 non-null int64
MachineHoursCurrentMeter    4739 non-null float64
UsageBand                   4031 non-null object
saledate                    11573 non-null datetime64[ns]
fiModelDesc                 11573 non-null object
fiBaseModel                 11573 non-null object
fiSecondaryDesc             8037 non-null object
fiModelSeries               1759 non-null object
fiModelDescriptor           2897 non-null object
ProductSize                 5743 non-null object
fiProductClassDesc          11573 non-null object
state                       11573 non-null object
ProductGroup                11573 non-null o

In [7]:
x1 = bull_valid.drop(['SalesID','saledate','MachineID','ModelID','datasource','auctioneerID','YearMade','MachineHoursCurrentMeter'],axis=1)
x2 = bull_valid.iloc[:,0:7]

In [8]:
# Fill missing values and categorize and numericalize the data

x2.fillna(value={'auctioneerID':0,'MachineHoursCurrentMeter':0},inplace=True)

def categorize(df):
    for n in df:
        df[n] = df[n].astype('category',inplace=True)

categorize(x1)        

x1.UsageBand.cat.set_categories(['Low','Medium','High'],ordered=True,inplace=True)

def numericalize(df):
    for n in df:
        df[n] = df[n].cat.codes + 1
        
        
numericalize(x1)

In [10]:
# merge x2 & x1 

x1['SalesID'] = bull_valid['SalesID']

x = pd.merge(x2,x1,on='SalesID',how='outer')

In [11]:
x.isna().sum()

SalesID                     0
MachineID                   0
ModelID                     0
datasource                  0
auctioneerID                0
YearMade                    0
MachineHoursCurrentMeter    0
UsageBand                   0
fiModelDesc                 0
fiBaseModel                 0
fiSecondaryDesc             0
fiModelSeries               0
fiModelDescriptor           0
ProductSize                 0
fiProductClassDesc          0
state                       0
ProductGroup                0
ProductGroupDesc            0
Drive_System                0
Enclosure                   0
Forks                       0
Pad_Type                    0
Ride_Control                0
Stick                       0
Transmission                0
Turbocharged                0
Blade_Extension             0
Blade_Width                 0
Enclosure_Type              0
Engine_Horsepower           0
Hydraulics                  0
Pushblock                   0
Ripper                      0
Scarifier 

In [13]:
# converting datetime into different features

x['DayOfYear'] = bull_valid.saledate.dt.dayofyear
x['WeekOfYear'] = bull_valid.saledate.dt.weekofyear
x['month_end'] = bull_valid.saledate.dt.is_month_end
x['month_start'] = bull_valid.saledate.dt.is_month_start
x['year'] = bull_valid.saledate.dt.year
x['month'] = bull_valid.saledate.dt.month


x.shape

(11573, 57)

In [14]:
valid_solution = pd.read_csv('ValidSolution.csv')

In [17]:
valid_solution.drop(['Usage'],axis=1,inplace=True)

In [18]:
valid_solution.shape

(11573, 2)

In [21]:
y_actual = np.log(valid_solution.SalePrice)

In [33]:
loaded_best_rf_model = load(filename='best_rf_model_1.joblib')

In [34]:
from math import sqrt


def print_scores(x,y_actual,y_pred,model):
    print(f'test score = ',model.score(x,y_actual))
    print(f'RMSL ERROR = ',sqrt(mean_squared_error(y_actual,y_pred)))

In [35]:
y_preds = loaded_best_rf_model.predict(x)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [36]:
print_scores(x,y_actual,y_preds,loaded_best_rf_model)

test score =  0.6053227820248752
RMSL ERROR =  0.4613906643395231


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
